<a href="https://colab.research.google.com/github/Esantomi/digital-humanities/blob/main/%EC%9D%B8%EB%AC%B8%EC%A7%80%EC%8B%9D%EC%B2%98%EB%A6%AC%EC%99%80%ED%86%B5%EA%B3%84/%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D/%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%8B%AC%ED%99%94_MECAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

https://github.com/jonghwanhyeon/python-mecab-ko

한국어 품사 태그 비교표:
https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [ ]:
# Mecab-ko를 설치합니다. (한글처리를 위한 library입니다.)
!pip install -v python-mecab-ko

Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /usr
Additional context:
user = False
home

In [ ]:
import mecab
mecab = mecab.MeCab()
# 나는 아메리카노가 좋아.
print(mecab.pos("나는 아메리카노가 좋아."))

[('나', 'NP'), ('는', 'JX'), ('아메리카노', 'NNP'), ('가', 'JKS'), ('좋', 'VA'), ('아', 'EF'), ('.', 'SF')]


# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 실습을 위한 역대 대통령 취임사: 노무현, 이명박, 박근혜, 문재인
## https://drive.google.com/open?id=1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK

rawdata_downloaded = drive.CreateFile({'id': '1VEdcJErjAjj2ul5gnS0-IgKrpGy0NEAK'})
rawdata_downloaded.GetContentFile('rawdata.tsv')

In [ ]:
import numpy as np
import pandas as pd
import codecs

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata.tsv', sep="\t")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,id,type,text
0,1,노무현,존경하는 국민 여러분. 오늘 저는 대한민국의 제16대 대통령에 취임하기 위해 이...
1,2,이명박,"존경하는 국민 여러분! 700만 해외동포 여러분, 이 자리에 참석하신 노무현, 김..."
2,3,박근혜,희망의 새 시대를 열겠습니다. 존경하는 국민 여러분! 700만 해외동포 여러분! ...
3,4,문재인,"존경하고 사랑하는 국민 여러분, 감사합니다. 국민 여러분의 위대한 선택에 머리 숙여..."


# 분석 프로세스

In [ ]:
# "원본데이터"를 대상으로 "mecab"를 사용해서 형태소 분석하기
from itertools import product
import numpy as np
import mecab
mecab = mecab.MeCab()

저장공간  = []
for index, row in 원본데이터.iterrows():# 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(mecab.pos(대상텍스트))  # mecab의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('존경', 'NNG'),
 ('하', 'XSV'),
 ('고', 'EC'),
 ('사랑', 'NNG'),
 ('하', 'XSV'),
 ('는', 'ETM'),
 ('국민', 'NNG'),
 ('여러분', 'NP'),
 (',', 'SC'),
 ('감사', 'NNG'),
 ('합니다', 'XSV+EF'),
 ('.', 'SF'),
 ('국민', 'NNG'),
 ('여러분', 'NP'),
 ('의', 'JKG'),
 ('위대', 'XR'),
 ('한', 'XSA+ETM'),
 ('선택', 'NNG'),
 ('에', 'JKB'),
 ('머리', 'NNG'),
 ('숙여', 'VV+EC'),
 ('깊이', 'MAG'),
 ('감사', 'NNG'),
 ('드립니다', 'VV+EF'),
 ('.', 'SF'),
 ('저', 'NP'),
 ('는', 'JX'),
 ('오늘', 'MAG'),
 ('대한민국', 'NNP'),
 ('제', 'XPN'),
 ('19', 'SN'),
 ('대', 'NNBC'),
 ('대통령', 'NNG'),
 ('으로서', 'JKB'),
 ('새로운', 'VA+ETM'),
 ('대한민국', 'NNP'),
 ('을', 'JKO'),
 ('향해', 'VV+EC'),
 ('첫', 'MM'),
 ('걸음', 'NNG'),
 ('을', 'JKO'),
 ('내딛', 'VV'),
 ('습니다', 'EF'),
 ('.', 'SF'),
 ('지금', 'MAG'),
 ('제', 'MM'),
 ('두', 'MM'),
 ('어깨', 'NNG'),
 ('는', 'JX'),
 ('국민', 'NNG'),
 ('여러분', 'NP'),
 ('으로부터', 'JKB'),
 ('부여', 'NNG'),
 ('받', 'VV'),
 ('은', 'ETM'),
 ('막중', 'XR'),
 ('한', 'XSA+ETM'),
 ('소', 'XPN'),
 ('명감', 'NNG'),
 ('으로', 'JKB'),
 ('무겁', 'VA'),
 ('습니다', 'EF'),
 ('.', 'SF'),
 ('

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,존경,NNG,문재인,1
1,하,XSV,문재인,1
2,고,EC,문재인,1
3,사랑,NNG,문재인,1
4,하,XSV,문재인,1
...,...,...,...,...
1502,습니다,EF,문재인,1
1503,.,SF,문재인,1
1504,감사,NNG,문재인,1
1505,합니다,XSV+EF,문재인,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[        0       1 type  count
 0      존경     NNG  노무현      1
 1       하     XSV  노무현      1
 2       는     ETM  노무현      1
 3      국민     NNG  노무현      1
 4     여러분      NP  노무현      1
 ...   ...     ...  ...    ...
 2543  합니다  XSV+EF  노무현      1
 2544    .      SF  노무현      1
 2545   감사     NNG  노무현      1
 2546  합니다  XSV+EF  노무현      1
 2547    .      SF  노무현      1
 
 [2548 rows x 4 columns],         0       1 type  count
 0      존경     NNG  이명박      1
 1       하     XSV  이명박      1
 2       는     ETM  이명박      1
 3      국민     NNG  이명박      1
 4     여러분      NP  이명박      1
 ...   ...     ...  ...    ...
 4188  입니다  VCP+EF  이명박      1
 4189    .      SF  이명박      1
 4190   감사     NNG  이명박      1
 4191  합니다  XSV+EF  이명박      1
 4192    .      SF  이명박      1
 
 [4193 rows x 4 columns],         0       1 type  count
 0      희망     NNG  박근혜      1
 1       의     JKG  박근혜      1
 2       새      MM  박근혜      1
 3      시대     NNG  박근혜      1
 4       를     JKO  박근혜      1
 ...   ...     .

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,존경,NNG,노무현,1
1,하,XSV,노무현,1
2,는,ETM,노무현,1
3,국민,NNG,노무현,1
4,여러분,NP,노무현,1
...,...,...,...,...
1502,습니다,EF,문재인,1
1503,.,SF,문재인,1
1504,감사,NNG,문재인,1
1505,합니다,XSV+EF,문재인,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,존경,NNG,노무현,1
1,하,XSV,노무현,1
2,는,ETM,노무현,1
3,국민,NNG,노무현,1
4,여러분,NP,노무현,1
...,...,...,...,...
1502,습니다,EF,문재인,1
1503,.,SF,문재인,1
1504,감사,NNG,문재인,1
1505,합니다,XSV+EF,문재인,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사      분류 
!    SF      박근혜     7
             이명박     8
     SY      박근혜     2
             이명박     1
'    SY      노무현    10
                    ..
힘    NNG     이명박     5
힘들   VA      문재인     1
힘쓰   VV      이명박     2
힘쓸   VV+ETM  이명박     1
힘차   VA      이명박     1
Name: 카운트, Length: 3151, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사     분류      
!   SF     박근혜    7
           이명박    8
    SY     박근혜    2
           이명박    1
'   SY     노무현   10
...             ...
힘   NNG    이명박    5
힘들  VA     문재인    1
힘쓰  VV     이명박    2
힘쓸  VV+ETM 이명박    1
힘차  VA     이명박    1

[3151 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')